In [29]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm

In [30]:
signals = pd.read_excel('data/gmo_analysis_data.xlsx', sheet_name = 'signals').set_index("date")
rf = pd.read_excel('data/gmo_analysis_data.xlsx', sheet_name = 'risk-free rate').set_index("date")
returns = pd.read_excel('data/gmo_analysis_data.xlsx', sheet_name = 'total returns').set_index("date")

In [31]:
signals.head()

,SPX D/P,SPX E/P,T-Note 10YR
date,,,
1996-12-31,0.019651,0.051592,0.06418
1997-01-31,0.018455,0.048704,0.06494
1997-02-28,0.018502,0.048434,0.06552
1997-03-31,0.019427,0.055559,0.06903
1997-04-30,0.018430,0.052318,0.06718


In [32]:
rf.head()

,TBill 3M
date,
1996-12-31,0.05171
1997-01-31,0.05147
1997-02-28,0.05220
1997-03-31,0.05322
1997-04-30,0.05233


In [33]:
returns.head()

,SPY,GMWAX,GMGEX
date,,,
1996-12-31,-0.023292,-0.022094,-0.013000
1997-01-31,0.061786,0.014735,0.034448
1997-02-28,0.009565,0.022265,0.012733
1997-03-31,-0.045721,-0.015152,-0.016441
1997-04-30,0.064368,-0.006731,0.000000


# 3. Forecast Regressions

## 3.1 Lagged regression

In [34]:
X = signals.shift(1)
rx = returns['SPY'] - rf['TBill 3M']
df = pd.concat([rx.rename("rx"), X], axis=1).dropna()

In [35]:
def run_regression(y, X):
    X = sm.add_constant(X)
    model = sm.OLS(y, X).fit()
    return model

In [36]:
model_dp = run_regression(df['rx'], df[['SPX D/P']])
print(model_dp.summary())

                            OLS Regression Results                            
Dep. Variable:                     rx   R-squared:                       0.103
Model:                            OLS   Adj. R-squared:                  0.100
Method:                 Least Squares   F-statistic:                     39.55
Date:                Sun, 16 Nov 2025   Prob (F-statistic):           9.72e-10
Time:                        11:30:03   Log-Likelihood:                 573.97
No. Observations:                 346   AIC:                            -1144.
Df Residuals:                     344   BIC:                            -1136.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0824      0.011     -7.315      0.0

In [37]:
model_ep = run_regression(df['rx'], df[['SPX E/P']])
print(model_ep.summary())

                            OLS Regression Results                            
Dep. Variable:                     rx   R-squared:                       0.019
Model:                            OLS   Adj. R-squared:                  0.016
Method:                 Least Squares   F-statistic:                     6.634
Date:                Sun, 16 Nov 2025   Prob (F-statistic):             0.0104
Time:                        11:30:03   Log-Likelihood:                 558.45
No. Observations:                 346   AIC:                            -1113.
Df Residuals:                     344   BIC:                            -1105.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0415      0.011     -3.689      0.0

In [38]:
model_three = run_regression(df['rx'], df[['SPX D/P','SPX E/P','T-Note 10YR']])
print(model_three.summary())

                            OLS Regression Results                            
Dep. Variable:                     rx   R-squared:                       0.184
Model:                            OLS   Adj. R-squared:                  0.176
Method:                 Least Squares   F-statistic:                     25.65
Date:                Sun, 16 Nov 2025   Prob (F-statistic):           5.47e-15
Time:                        11:30:03   Log-Likelihood:                 590.25
No. Observations:                 346   AIC:                            -1172.
Df Residuals:                     342   BIC:                            -1157.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const          -0.0133      0.016     -0.831      

In [39]:
print("R² (DP):", model_dp.rsquared)
print("R² (EP):", model_ep.rsquared)
print("R² (Three vars):", model_three.rsquared)
print("Adj R² (DP):", model_dp.rsquared_adj)
print("Adj R² (EP):", model_ep.rsquared_adj)
print("Adj R² (Three vars):", model_three.rsquared_adj)


R² (DP): 0.10310646336284202
R² (EP): 0.01892091939804519
R² (Three vars): 0.18365468768848736
Adj R² (DP): 0.10049921470982703
Adj R² (EP): 0.016068945326527917
Adj R² (Three vars): 0.17649376389628113


## 3.2 Trading strategy from forecasts

In [40]:
df['f_dp'] = model_dp.predict(sm.add_constant(df[['SPX D/P']]))
df['f_ep'] = model_ep.predict(sm.add_constant(df[['SPX E/P']]))
df['f_three'] = model_three.predict(
    sm.add_constant(df[['SPX D/P','SPX E/P','T-Note 10YR']])
)

df['w_dp'] = 100 * df['f_dp']
df['w_ep'] = 100 * df['f_ep']
df['w_three'] = 100 * df['f_three']

spy = returns['SPY']        
df['rx_dp'] = df['w_dp'] * spy
df['rx_ep'] = df['w_ep'] * spy
df['rx_three'] = df['w_three'] * spy

In [41]:
def max_drawdown(r):
    cum = (1 + r).cumprod()
    peak = cum.cummax()
    dd = (cum - peak) / peak
    return dd.min()

In [42]:
def alpha_beta_ir(r_strategy, r_market):
    data = pd.concat([r_strategy, r_market], axis=1).dropna()
    y = data.iloc[:,0]
    x = data.iloc[:,1]
    X = sm.add_constant(x)
    res = sm.OLS(y, X).fit()
    alpha_m = res.params['const']        
    beta = res.params.iloc[1]
    resid_vol_m = res.resid.std()
    ir_m = alpha_m / resid_vol_m          
    return alpha_m, beta, ir_m

In [43]:
for name in ['dp', 'ep', 'three']:
    r = df[f'rx_{name}'].dropna()
    m = spy.loc[r.index]

    mean_m = r.mean()
    vol_m = r.std()
    sharpe_m = mean_m / vol_m

    mean_a = mean_m * 12
    vol_a = vol_m * np.sqrt(12)
    sharpe_a = sharpe_m * np.sqrt(12)

    dd = max_drawdown(r)

    alpha_m, beta, ir_m = alpha_beta_ir(r, m)
    alpha_a = alpha_m * 12
    ir_a = ir_m * np.sqrt(12)

    print(f'=== {name.upper()} strategy ===')
    print(f'Mean (annual)        : {mean_a:.4f}')
    print(f'Volatility (annual)  : {vol_a:.4f}')
    print(f'Sharpe (annual)      : {sharpe_a:.4f}')
    print(f'Max drawdown         : {dd:.4f}')
    print(f'Alpha (annual)       : {alpha_a:.4f}')
    print(f'Beta                 : {beta:.4f}')
    print(f'Info ratio (annual)  : {ir_a:.4f}')
    print()

=== DP strategy ===
Mean (annual)        : -0.0713
Volatility (annual)  : 0.3603
Sharpe (annual)      : -0.1978
Max drawdown         : -0.9807
Alpha (annual)       : 0.0353
Beta                 : -0.9997
Info ratio (annual)  : 0.1084

=== EP strategy ===
Mean (annual)        : -0.1173
Volatility (annual)  : 0.2198
Sharpe (annual)      : -0.5336
Max drawdown         : -0.9845
Alpha (annual)       : 0.0109
Beta                 : -1.2014
Info ratio (annual)  : 0.0909

=== THREE strategy ===
Mean (annual)        : -0.0536
Volatility (annual)  : 0.4093
Sharpe (annual)      : -0.1310
Max drawdown         : -0.9893
Alpha (annual)       : 0.0698
Beta                 : -1.1568
Info ratio (annual)  : 0.1892



## 3.3 Risk characteristics

In [44]:
def var_5(r):
    return r.quantile(0.05)  

series_dict = {
    'strat_dp' : df['rx_dp'],
    'strat_ep' : df['rx_ep'],
    'market'   : returns['SPY'],
    'GMO'      : returns['GMWAX'], 
}

for name, r in series_dict.items():
    print(name, "VaR_5% =", var_5(r.dropna()))

strat_dp VaR_5% = -0.1655603788948047
strat_ep VaR_5% = -0.11026119498312509
market VaR_5% = -0.07438318192290026
GMO VaR_5% = -0.04036908917480489


In [45]:
start, end = '2000-01-01', '2011-12-31'
rx_dp_0011 = df['rx_dp'].loc[start:end]
rf_0011 = rf['TBill 3M'].loc[start:end]

excess_dyn = rx_dp_0011 - rf_0011  
print("Mean excess (dyn - rf), 2000–2011:", excess_dyn.mean())

Mean excess (dyn - rf), 2000–2011: -0.01293236302592116


Since the mean excess is negative, we see that the dynamic portfolio underperformed the risk-free rate from 2000 - 2011.

In [46]:
print("DP:  # periods with negative forecast =", (df['f_dp'] < 0).sum())
print("EP:  # periods with negative forecast =", (df['f_ep'] < 0).sum())
print("3-var: # periods with negative forecast =", (df['f_three'] < 0).sum())

DP:  # periods with negative forecast = 306
EP:  # periods with negative forecast = 331
3-var: # periods with negative forecast = 230


The dynamic strategy clearly takes on extra risk. Its 5% VaR is more negative than both the market and GMO, and its drawdowns are larger. Because the portfolio weight scales directly with the return forecast, the strategy naturally creates periods of high leverage, which amplifies both gains and losses. Overall, the timing rule increases risk rather than reducing it.

# 4. OOS Forecasting

In [47]:
rx = returns['SPY'] - rf['TBill 3M']
X = signals[['SPX D/P', 'SPX E/P']].shift(1)
oos = pd.concat([rx.rename('rx'), X], axis=1).dropna()

In [48]:
e_fore = []
e_null = []

X_full = sm.add_constant(oos[['SPX D/P', 'SPX E/P']], has_constant='add')

for i in range(59, len(oos) - 1):  
    y_train = oos['rx'].iloc[:i+1]
    X_train = X_full.iloc[:i+1]

    model = sm.OLS(y_train, X_train).fit()

    x_t = X_full.iloc[i+1:i+2]     
    r_hat = model.predict(x_t).iloc[0]

    r_real = oos['rx'].iloc[i+1]

    e_fore.append(r_real - r_hat)

    r_bar = oos['rx'].iloc[:i+1].mean()
    e_null.append(r_real - r_bar)

## 4.1 OOS R^2

In [49]:
e_fore = np.array(e_fore)
e_null = np.array(e_null)

R2_OOS = 1 - (e_fore**2).sum() / (e_null**2).sum()
print("OOS R^2 =", R2_OOS)

OOS R^2 = 0.0847892004558144


Yes, the forecasting strategy produced a positive out-of-sample R^2. This means the DP+EP model forecasts SPY returns better than the historical-mean (null) model, though the improvement is modest.

## 4.2 Trading Strategy from forecasts

In [55]:
w_oos = 100 * oos_forecasts
r_oos = w_oos * oos.loc[oos_idx, 'rx']

mean_oos = r_oos.mean()
vol_oos = r_oos.std()
sharpe_oos = mean_oos / vol_oos
dd_oos = max_dd(r_oos)

print("OOS mean (monthly):", mean_oos)
print("OOS vol (monthly):", vol_oos)
print("OOS Sharpe (monthly):", sharpe_oos)
print("OOS max drawdown:", dd_oos)

r_in = df['rx_three'].dropna()
in_sharpe = r_in.mean() / r_in.std()
in_dd = max_dd(r_in)

print("In-sample Sharpe (monthly):", in_sharpe)
print("In-sample DD:", in_dd)

OOS mean (monthly): 0.01772236267880823
OOS vol (monthly): 0.07479569590288486
OOS Sharpe (monthly): 0.23694361640566916
OOS max drawdown: -0.7475155478110145
In-sample Sharpe (monthly): -0.037803813411602966
In-sample DD: -0.9892829241937832


The OOS strategy performs substantially better than the in sample version. Its monthly Sharpe ratio is positive and meaningfully higher, and its drawdown is less severe than the in sample strategy, suggesting that the rolling OOS forecasts generate a more stable and better-behaved trading rule than the overfit in-sample model.

## 4.3 Risk Characteristics

In [56]:
def var_5(r):
    return r.quantile(0.05)

print("OOS VaR_5% =", var_5(r_oos))

start, end = '2000-01-01', '2011-12-31'
r_oos_0011 = r_oos.loc[start:end]
rf_0011 = rf['TBill 3M'].loc[start:end]
print("OOS mean excess 2000–2011 =", (r_oos_0011 - rf_0011).mean())

print("# periods with negative OOS forecast =", (oos_forecasts < 0).sum())

OOS VaR_5% = -0.08010075902880208
OOS mean excess 2000–2011 = -0.0010952452695693342
# periods with negative OOS forecast = 265


The OOS strategy remains risky: its 5% VaR is still materially negative, and it underperforms the risk-free rate on average during 2000–2011. It also produces a large number of negative expected-return forecasts, indicating that even the point-in-time version of the model continues to load into unfavorable market conditions.